In [2]:
from agential.cog.agent.reflexion import ReflexionReActAgent

In [3]:
from agential.cog.functional.expel import gather_experience
import warnings
warnings.filterwarnings('ignore')
from langchain_openai import ChatOpenAI

import dotenv
dotenv.load_dotenv()
from agential.cog.prompts.benchmark.hotpotqa import (
    HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
)
from agential.cog.prompts.agent.reflexion import (
    REFLEXION_REACT_INSTRUCTION_HOTPOTQA,
    REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
)
llm = ChatOpenAI()
reflexion_react_agent = ReflexionReActAgent(llm=llm, mode={"qa": "hotpotqa"}, max_trials=3)

questions = ["VIVA Media AG changed it's name in 2004. What does their new acronym stand for?",
 'Which of Jonny Craig and Pete Doherty has been a member of more bands ?',
 'Where was the first governor after the The Missouri Compromise from?',
 'The creator of "Wallace and Gromit" also created what animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes? ',
 "Woman's Era and Naj are what kind of magazines?"]
keys = ['Gesellschaft mit beschränkter Haftung',
 'Jonny" Craig',
 'Bath, Maine',
 'Creature Comforts',
 'fortnightly women interest magazine']

experiences = gather_experience(
    reflexion_react_agent,
    questions=questions,
    keys=keys,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    prompt=REFLEXION_REACT_INSTRUCTION_HOTPOTQA,
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA,
    patience=3
)

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [9]:
import joblib
joblib.dump(experiences, "joblib_file.joblib")

['joblib_file.joblib']